In [55]:
from dataclasses import dataclass, field, fields, asdict
import yaml
from typing import List
from collections import namedtuple, defaultdict, Counter
import random
import pandas as pd
import numpy as np
import csv
import sys
from openpyxl import load_workbook, Workbook
import pprint
import os
import logging
import shutil

In [ ]:
#DEFINE CLASSES
#dataclass for model properties
@dataclass
class ModelProperty:
    name: str = field(default = "")
    desc: str = field(default = "")
    value_type: str = field(default = "")
    value_list: List[str] = field(default_factory=list)
    synthetic_value_list: List[str] = field(default_factory=list)
    units: List[str] = field(default_factory=list)
    pattern: str = field(default = "")
    url: str = field(default = "")
    req: bool = field(default = False)
    private: bool = field(default = False)
    minimum: int = field(default = 0)
    maximum: int = field(default = 1000)
    exclusiveMinimum:  str = field(default = "")
    exclusiveMaximum:  str = field(default = "")
    #dist: str = field(default = 'Uniform')
    weights: List[int]= field(default_factory=list)
    
    def get_weighted_values(self):
        self.weights = []
        
        if Node_Props["Minage"] != '' and self.name  in ['age_at_enrollment', 'age_at_first_cancer_diagnosis']:
            self.minimum = eval(Node_Props["Minage"])
            #print(f"{self.name}, : {self.minimum}")
        if Node_Props["Maxage"] != '' and self.name  in ['age_at_enrollment', 'age_at_first_cancer_diagnosis']:
            self.maximum = eval(Node_Props["Maxage"])
        
        if self.name in weight_dict:
            curr_dict = weight_dict[self.name]   
            if True in curr_dict: curr_dict.update({"Yes": curr_dict.pop(True)})
            if False in curr_dict: curr_dict.update({"No": curr_dict.pop(False)})
            
            if (len(self.value_list) - len(curr_dict)) > 0:
                missing_pct = round((1-sum(curr_dict.values())) / (len(self.value_list) - len(curr_dict)) , 3)
            else:
                missing_pct = 0
            weight_list = []
            for curr_value in self.value_list:
                if curr_value in curr_dict:
                    self.weights.append(curr_dict[curr_value])
                else:
                    self.weights.append(missing_pct)
        if sum(self.weights) == 0:
            self.weights = []

    
    def emit_value(self):
        #relationship_node_dict[node_type]['parent_id_field'][index][0]
        if self.name  in ['participant_race'] and "multi_race_opt" in weight_dict:
            uni_choice = random.choices([1,2], weights=weight_dict["multi_race_opt"].values())[0]
        else:
            uni_choice = 1
        
        curr_choice = 1
        
        property_data_value = ""
        output_value = []
        while curr_choice <= uni_choice:
            self.get_weighted_values()
            if self.name  in ['cancer_diagnosis_primary_site'] and cancer_location_type  == 'ICD-O-3':  #use codes
                list_of_lists = cancer_location_df["ICD-0-3 Code"].values.tolist()
                self.value_list = [item for sublist in list_of_lists for item in sublist]
            if self.name  in ['cancer_diagnosis_disease_morphology'] and cancer_histology_type  == 'ICD-O-3':  #use codes
                list_of_lists = cancer_histology_df["ICD-0-3 Code"].values.tolist()
                self.value_list = [item for sublist in list_of_lists for item in sublist]

            
            #print(f"{self.name} has weights {self.value_list}")
            #if  self.name  in ['cancer_diagnosis_primary_site']:  #use translantion file instead
            #    self.value_list = cancer_location_df["ICD-0-3 Code"].values.flatten().tolist()
            
            if self.synthetic_value_list:
                if len(self.weights) == 0:  #no weights provided
                    property_data_value = random.choice(self.synthetic_value_list)
                else:
                    property_data_value = random.choices(self.synthetic_value_list, weights=self.weights)[0]
                #return property_data_value
            if self.value_list:
                if len(self.weights) == 0:  #no weights provided
                    property_data_value =  random.choice(self.value_list)
                else:
                    property_data_value = random.choices(self.value_list, weights=self.weights)[0]
                #return property_data_value
            if self.value_type == 'string':
                base_list = ["a_bene_placito",
                             "barba_crescit_caput_nescit",
                             "cacatum_non_est_pictum",
                             "damnant_quod_non_intellegunt","e_causa_ignota",
                             "faber_est_suae_quisque_fortunae",
                             "Gallia_est_omnis_divisa_in_partes_tres","haec_olim_meminisse_iuvabit",
                             "id_quod_plerumque_accidit","imperium_in_imperio","labor_ipse_voluptas",
                             "Macte_animo_Generose_puer_sic_itur_ad_astra","nanos_gigantum_humeris_insidentes",
                             "nascentes_morimur_finisque_ab_origine_pendet","O_Tite_tute_Tati_tibi_tanta_tyranne_tulisti",
                             "Obedientia_civium_urbis_felicitas","pace_tua","saltus_in_demonstrando",
                             "salus_in_arduis","sapiens_qui_prospicit","scientia_et_labor","scientia_et_sapientia",
                             "scientia_imperii_decus_et_tutamen","scientia,_aere_perennius","scientiae_cedit_mare",
                             "scientiae_et_patriae"]
                property_data_value = random.choice(base_list)
                #return property_data_value
            if self.value_type == 'number':
                if type(self.minimum) == float and type(self.maximum) == float:
                    property_data_value = random.uniform(self.minimum, self.maximum)
                elif type(self.minimum) == int and type(self.maximum) == int:
                    property_data_value = random.uniform(self.minimum, self.maximum)
                else:
                    property_data_value = random.uniform(10.0,1000.0)
                if "age" in self.name:
                    property_data_value = round(property_data_value, 0)  #age is a whole number in days
                else:
                    property_data_value = round(property_data_value, 2)
                #return property_data_value
            if self.value_type == 'boolean':
                property_data_value = random.choice([True, False])
                #return property_data_value
            if self.value_type == 'integer':
                if type(self.minimum) == int and type(self.maximum) == int:
                    property_data_value = random.randint(self.minimum, self.maximum)
                else:
                    property_data_value = random.randint(10,1000)
                property_data_value = round(property_data_value, 0)
            curr_choice = curr_choice + 1
            output_value = output_value + [property_data_value]
        
        output_value = list(set(output_value)) #get unique records
        if len(output_value) > 1:
            remove_list = ["Not Reported", "Unknown", "Not allowed to collect"]
            for not_allowed in remove_list:
                if not_allowed in output_value:
                    output_value.remove(not_allowed)
                    if len(output_value) == 1:
                        break
            output_value.sort()  #sort values
            output_value = " | ".join(output_value)  #convert to string
        else:
            output_value = output_value[0]
           
        return output_value  #property_data_value

In [57]:
#dataclass for model nodes
@dataclass
class ModelNode:
    name: str = field(default = "")
    properties: List[ModelProperty] = field(default_factory=list)

In [58]:
#dataclass for the ends of a model relationship
@dataclass
class ModelEnds:
    source_node: ModelNode
    destination_node: ModelNode
    multiplicity: str = field(default = "")

In [59]:
#dataclass for model relationships
@dataclass
class ModelEdge:
    name: str
    ends_list: List[ModelEnds] = field(default_factory=list)
    properties_list: List[ModelProperty] = field(default_factory=list)

In [60]:
#dataclass for mock data nodes
@dataclass
class DataNode:
    node_id: str
    parent_node_id_list: list
    child_node_id_list: list
    node_type: str
    node_attributes: dict

In [61]:
#dataclass for mock data relationships
@dataclass
class DataEdge:
    edge_id: str
    edge_type: str
    edge_attributes: dict
    source_node: DataNode
    destination_node: DataNode

In [62]:
@dataclass
class Graph:
    dict_of_data_nodes: defaultdict(list)
    dict_of_data_edges: {}
    
    def print_data(self, node_type = 'all'):
        if node_type == 'all':
            for node_type_key in data_graph.dict_of_data_nodes:
                node_values_dict = defaultdict(list)
                df = pd.DataFrame()
                for node in data_graph.dict_of_data_nodes[node_type_key]:
                    node_values_dict['type'].append(node.node_type)
                
                if  node.parent_node_id_list:
                    node_values_dict['parent_id'].append(node.parent_node_id_list[0])

                node_values_dict['node_id'].append(node.node_id)
                for node_prop in node.node_attributes:
                    node_values_dict[node_prop].append(node.node_attributes[node_prop])
                
                for node_values_key in node_values_dict:
                    df[node_values_key] = node_values_dict[node_values_key]
                
                file_name = node_type_key + ".csv"
                df.to_csv(file_name)
            
            return
        else:
            print(data_graph.dict_of_data_nodes)
            if node_type in data_graph.dict_of_data_nodes:
                node_values_dict = defaultdict(list)
                df = pd.DataFrame()
                for node in data_graph.dict_of_data_nodes[node_type]:
                    node_values_dict['type'].append(node.node_type)
                
                if  node.parent_node_id_list:
                    node_values_dict['parent_id'].append(node.parent_node_id_list[0])
                
                node_values_dict['node_id'].append(node.node_id)
                
                for node_prop in node.node_attributes:
                    node_values_dict[node_prop].append(node.node_attributes[node_prop])
                
                print(node_values_dict)
                for node_values_key in node_values_dict:
                    df[node_values_key] = node_values_dict[node_values_key]
                
                file_name = node_type_key + ".csv"
                df.to_csv(file_name)
                return
            else:
                print("node type not found in graph.")
                return
    
    def fill_graph(self, listOfProps, model_nodes_dict, model_props_dict): # , cancer_dict, tumor_dict):
        for node_type in self.dict_of_data_nodes:
            listOfNodeProps = model_nodes_dict[node_type].properties            
            listOfDataNodes = self.dict_of_data_nodes[node_type]
            try:
                print(f"Populating node_type for {node_type}")                
                for data_node in listOfDataNodes:
                    for prop in listOfNodeProps:
                        if prop.name in listOfProps[data_node.node_type]:
                            if model_props_dict[prop.name].emit_value() != None:
                                data_node.node_attributes[prop.name] = model_props_dict[prop.name].emit_value()
                                # print(data_node.node_attributes[prop.name])
                            else:
                                # if the value type can no be identified, then change type to string
                                base_string_list = ["a_bene_placito",
                                "barba_crescit_caput_nescit",
                                "cacatum_non_est_pictum",
                                "damnant_quod_non_intellegunt","e_causa_ignota",
                                "faber_est_suae_quisque_fortunae",
                                "Gallia_est_omnis_divisa_in_partes_tres","haec_olim_meminisse_iuvabit",
                                "id_quod_plerumque_accidit","imperium_in_imperio","labor_ipse_voluptas",
                                "Macte_animo_Generose_puer_sic_itur_ad_astra","nanos_gigantum_humeris_insidentes",
                                "nascentes_morimur_finisque_ab_origine_pendet","O_Tite_tute_Tati_tibi_tanta_tyranne_tulisti",
                                "Obedientia_civium_urbis_felicitas","pace_tua","saltus_in_demonstrando",
                                "salus_in_arduis","sapiens_qui_prospicit","scientia_et_labor","scientia_et_sapientia",
                                "scientia_imperii_decus_et_tutamen","scientia,_aere_perennius","scientiae_cedit_mare",
                                "scientiae_et_patriae"]
                                property_data_string_value = random.choice(base_string_list)
                                data_node.node_attributes[prop.name] = property_data_string_value
            except Exception as e:
                print("error found")
                print(prop.name)
                display_error_line(e)
        return
    
    def get_dict_of_data_nodes(self):
        return self.dict_of_data_nodes
    
    def get_dict_of_data_edges(self):
        return self.dict_of_data_edges
    
    def get_in_degree(self, input_node_id):
        for key in self.dict_of_data_nodes:
            for node in dict_of_data_nodes[key]:
                if node.node_id == input_node_id:
                    return len(node.parent_node_id_list)
    
    def get_out_degree(self, input_node_id):
        for key in self.dict_of_data_nodes:
            for node in dict_of_data_nodes[key]:
                if node.node_id == input_node_id:
                    return len(node.child_node_id_list)
    
    def summary(self):
        summary = {}
        summary['Nodes Summary'] = {}
        summary['Edges Summary']  = {}
        
        for node_type in self.dict_of_data_nodes:
            node_count = len(dict_of_data_nodes[node_type])
            summary['Nodes Summary'].update({node_type: node_count})
        
        edge_type_list = []
        for edge in dict_of_data_edges.values():
            edge_type_list.append(edge.edge_type)
        edge_type_counter = Counter(edge_type_list)
        for edge_type, edge_type_count in edge_type_counter.items():
            summary['Edges Summary'].update({edge_type: edge_type_count})
        
        return summary
            

In [63]:
def display_error_line(ex):
    trace = []
    tb = ex.__traceback__
    while tb is not None:
        trace.append({"filename": tb.tb_frame.f_code.co_filename, "name": tb.tb_frame.f_code.co_name, "lineno": tb.tb_lineno})
        tb = tb.tb_next
    print(str({'type': type(ex).__name__, 'message': str(ex), 'trace': trace}))

In [64]:
######BEGIN READ SECTION######
print('BEGIN READ SECTION')
dict_of_model_properties = {}
dict_of_model_nodes = {}
model_graph = {}

BEGIN READ SECTION


In [65]:
file_sep = os.path.sep

In [66]:
configuration_files = './popsci-mock-data-config-files.yaml'
with open(configuration_files) as f:
    configuration_files = yaml.load(f, Loader=yaml.FullLoader)

In [67]:
configuration_files

{'NODE_FILE': '.\\popsci-model.yml',
 'PROP_FILE': '.\\popsci-model-props.yml',
 'DATA_SPEC_FILE': '.\\popsci-mock-data-specs_v2.yaml',
 'ID_FILE': '.\\props-popsci.yml',
 'SYNTHETIC_DATA_FILE': '.\\cds-synthetic_data_values.xlsx',
 'ICDO_CONV_FILE': '.\\sitetype.icdo3.d20220429.xlsx',
 'LOCATION_CONV_FILE': '.\\cancer_site_location.yaml',
 'HISTOLOGY_CONV_FILE': '.\\cancer_histology.yaml',
 'STUDY_FILE': '.\\Study_Files\\',
 'OUTPUT_FOLDER': '.\\Mock_Data_Output\\'}

In [68]:
cancer_conv_file = configuration_files['ICDO_CONV_FILE']
workbook = load_workbook(cancer_conv_file, read_only=True, keep_links=False)

sheet_name = workbook.sheetnames

cancer_data = pd.read_excel(cancer_conv_file, sheet_name = sheet_name[0])

In [69]:
#site = data[index][1]
#sub_region = data[index][2]
#Histology Description = data[index][3]
#code = data[index][4]
#translation = data[index][5]

In [70]:
#tumor_dict =   {'0': 'Benign', 
#                '1': 'Uncertain whether benign or malignant',
#                '2': 'Carcinoma in situ',
#                '3': 'Malignant, primary site',
#                '6': 'Malignant, metastatic site',
#                '9': 'Malignant, uncertain whether primary or metastatic site'}

In [71]:
#READ MODEL FILES AND FILE WITH SYNTHETIC VALUES
#FOR BENTO
NODE_FILE = configuration_files['NODE_FILE']
PROP_FILE = configuration_files['PROP_FILE']
SPEC_FILE = configuration_files['DATA_SPEC_FILE']
SYNTHETIC_DATA_FILE = configuration_files['SYNTHETIC_DATA_FILE']

cancer_location_conv = configuration_files['LOCATION_CONV_FILE']
cancer_histology_conv = configuration_files['HISTOLOGY_CONV_FILE']

In [72]:
synthetic_values_df = pd.read_excel(io = SYNTHETIC_DATA_FILE,
                        sheet_name = "Sheet1",
                        engine = "openpyxl",
                        keep_default_na = False)

In [73]:
#cancer_term_file = r'C:\Users\breadsp2\Desktop\Current_Model_Files\cancer_site_location.yaml'
with open(cancer_location_conv) as f:
    cancer_term_file = yaml.load(f, Loader=yaml.FullLoader)

all_terms = pd.DataFrame()
for curr_location in cancer_term_file["CancerLocation"]:
    x = pd.DataFrame.from_dict(cancer_term_file["CancerLocation"][curr_location], orient = 'index')
    x["Primary Site"] = curr_location
    x.reset_index(inplace=True)
    
    all_terms = pd.concat([all_terms, x])
    
all_terms.columns = [["Sub Site", "ICD-0-3 Code", "Primary Site"]]
cancer_location_df = all_terms [[ "Primary Site", "Sub Site", "ICD-0-3 Code"]]

In [74]:
list_of_lists = cancer_location_df["ICD-0-3 Code"].values.tolist()
flat_list = [item for sublist in list_of_lists for item in sublist]


In [75]:
#cancer_term_file = r'C:\Users\breadsp2\Desktop\Current_Model_Files\cancer_site_location.yaml'
with open(cancer_histology_conv) as f:
    cancer_term_file = yaml.load(f, Loader=yaml.FullLoader)

all_terms = pd.DataFrame()
for curr_location in cancer_term_file["CancerHistology"]:
    x = pd.DataFrame.from_dict(cancer_term_file["CancerHistology"][curr_location], orient = 'index')
    x["Primary Site"] = curr_location
    x.reset_index(inplace=True)
    
    all_terms = pd.concat([all_terms, x])
    
all_terms.columns = [["Sub Type", "ICD-0-3 Code", "Primary Morphology"]]
cancer_histology_df = all_terms [[ "Primary Morphology", "Sub Type", "ICD-0-3 Code"]]

In [76]:
with open(SPEC_FILE) as f:
    data_spec= yaml.load(f, Loader=yaml.FullLoader)

    #IncludeProperties = spec_data['IncludeProperties']

In [77]:
weighted_data = data_spec['WeightProperties']    
weight_dict = {}
for weight_name in weighted_data.keys():
    temp_dict = {}
    [temp_dict.update(i) for i in weighted_data[weight_name]]
    weight_dict.update({weight_name: temp_dict})

In [78]:
cancer_location_type = data_spec["ICD_CodePropertes"]["location"]
cancer_histology_type = data_spec["ICD_CodePropertes"]["histology"]

In [79]:
#with open(WEIGHT_FILE) as f:
#    weighted_data = yaml.load(f, Loader=yaml.FullLoader)

In [80]:
with open(PROP_FILE) as f:
    property_data = yaml.load(f, Loader=yaml.FullLoader)
    for property_name in property_data['PropDefinitions'].keys():
        try:
            property_value_type = property_data['PropDefinitions'][property_name]['Type']
        except:
            try:
                #property_value_type = property_data['PropDefinitions'][property_name]['Enum']
                property_value_type = property_data['PropDefinitions'][property_name]
            except:
                property_value_type = 'string'
                print(property_name)
        
        name = property_name
        desc = ""
        req= ""
        value_type = ""
        value_list = []
        synthetic_value_list = []
        units = []
        private = ""
        pattern = ""
        url = ""
        minimum = 0
        maximum = 1000
        exclusiveMinimum = ""
        exclusiveMaximum = ""
        #dist = 'Uniform'
        #weights = []

        if type(property_value_type) is str:
            name = property_name
            value_type = property_value_type
            if 'Desc' in property_data['PropDefinitions'][property_name]:
                desc = property_data['PropDefinitions'][property_name]['Desc']
            if 'Req' in property_data['PropDefinitions'][property_name]:
                req = property_data['PropDefinitions'][property_name]['Req']
            if 'Private' in property_data['PropDefinitions'][property_name]:
                private = property_data['PropDefinitions'][property_name]['Private']
            if 'minimum' in property_data['PropDefinitions'][property_name]:
                minimum = property_data['PropDefinitions'][property_name]['minimum']
            if 'maximum' in property_data['PropDefinitions'][property_name]:
                maximum = property_data['PropDefinitions'][property_name]['maximum']

                
        if type(property_value_type) is list:
            value_type = "list"
            value_list = property_value_type['Enum']
            #add section on reading the url to create a value list if property_value_type contains a url.
            if 'Desc' in property_data['PropDefinitions'][property_name]:
                desc = property_data['PropDefinitions'][property_name]['Desc']
            if 'Req' in property_data['PropDefinitions'][property_name]:
                req = property_data['PropDefinitions'][property_name]['Req']
            if 'Private' in property_data['PropDefinitions'][property_name]:
                private = property_data['PropDefinitions'][property_name]['Private']
            #if 'dist' in property_data['PropDefinitions'][property_name]:
            #    dist = property_data['PropDefinitions'][property_name]['dist']
            #if 'weights' in property_data['PropDefinitions'][property_name]:
            #    weights = property_data['PropDefinitions'][property_name]['weights']

                
        if type(property_value_type) is dict:
            if 'Desc' in property_data['PropDefinitions'][property_name]:
                desc = property_data['PropDefinitions'][property_name]['Desc']
            if 'value_type' in property_value_type:
                value_type = property_value_type['value_type']
            if "Enum" in property_value_type:
                value_list = property_value_type["Enum"]
            if 'units' in property_value_type:
                units = property_value_type['units']
            if 'pattern' in property_value_type:
                pattern = property_value_type['pattern']
                value_type = "regex"
            if 'Req' in property_data['PropDefinitions'][property_name]:
                req = property_data['PropDefinitions'][property_name]['Req']
            if 'Private' in property_data['PropDefinitions'][property_name]:
                private = property_data['PropDefinitions'][property_name]['Private']
            if 'minimum' in property_data['PropDefinitions'][property_name]:
                minimum = property_data['PropDefinitions'][property_name]['minimum']
            if 'maximum' in property_data['PropDefinitions'][property_name]:
                maximum = property_data['PropDefinitions'][property_name]['maximum']
            
        dict_of_model_properties[property_name] = ModelProperty(name = name, desc = desc, 
                                                                    value_type = value_type, value_list = value_list,
                                                                    units = units, url = url, req = req, private = private, minimum = minimum, maximum = maximum,
                                                                    exclusiveMinimum = exclusiveMinimum, exclusiveMaximum = exclusiveMaximum, synthetic_value_list = synthetic_value_list)
                                                                    # ,dist = dist, weights = weights)
        

In [81]:
with open(NODE_FILE) as f:
    node_data = yaml.load(f, Loader=yaml.FullLoader)

In [82]:
nodes = node_data['Nodes']

for node_name in nodes.keys():
    #print(node_name, nodes[node_name]['Props'])
    if nodes[node_name]['Props']:
        try:
            property_list = [dict_of_model_properties[property_name] for property_name in nodes[node_name]['Props']]
        except Exception:
            print(f"property: {property_name} has issues for node: {node_name}")
            property_list = []
    else:
        property_list = []
    dict_of_model_nodes[node_name] = ModelNode(name = node_name, properties = property_list)

property: data_file_access_control has issues for node: study_personnel


In [83]:
edges = node_data['Relationships']

for edge_name in edges.keys():
    Ends_list = []
    Property_list = []
    edge_multiplicity = edges[edge_name]['Mul']
    
    for edge_pair in edges[edge_name]['Ends']:
        source_node = edge_pair['Src']
        destination_node = edge_pair['Dst']
        if 'Mul' in edge_pair:
            edge_multiplicity = edge_pair['Mul']
        Ends_list.append(ModelEnds(source_node = dict_of_model_nodes[source_node], destination_node = dict_of_model_nodes[destination_node], multiplicity = edge_multiplicity))
        
    
    if 'Props' in edges[edge_name] and edges[edge_name]['Props']:
        property_list = [dict_of_model_properties[property_name] for property_name in edges[edge_name]['Props']]
    else:
        property_list = []
    
    model_graph[edge_name] = ModelEdge(name = edge_name, ends_list = Ends_list, properties_list = Property_list)
#END READ MODEL FILES
print('END READ SECTION')
######END READ SECTION######

END READ SECTION


In [84]:
######BEGIN SPAWN SECTION######
print('BEGIN SPAWN SECTION')
dict_of_data_nodes = defaultdict(list)
dict_of_data_edges = {}

BEGIN SPAWN SECTION


In [85]:
#READ DATA SPECS FILE
#FOR BENTO
DATA_SPEC_FILE = configuration_files['DATA_SPEC_FILE']

In [86]:
with open(DATA_SPEC_FILE) as f:
    data_spec = yaml.load(f, Loader=yaml.FullLoader)

In [87]:
ID_FILE = configuration_files['ID_FILE']
with open(ID_FILE) as f:
    id_field_data = yaml.load(f, Loader=yaml.FullLoader)

In [88]:
ICDO_CONV_FILE = configuration_files['ICDO_CONV_FILE']
with open(ICDO_CONV_FILE) as f:
    id_conv_file = pd.read_excel(ICDO_CONV_FILE)

In [89]:
def import_study_file(study_path):
    data = []
    with open(study_path, 'r') as tsvfile:
        tsvreader = csv.reader(tsvfile, delimiter='\t')
        for row in tsvreader:
            data.append(row)
    study_df = pd.DataFrame(columns = data[0])
    study_df.loc[len(study_df)] = data[1]
    return study_df

In [90]:
includeNodes = data_spec['IncludeNodes']
Node_Props = data_spec['MockDataProps']
Output_Folder = configuration_files['OUTPUT_FOLDER'] + "Study_" + data_spec['MockDataProps']["StudyName"] 


In [91]:
#Create head data node object
for head_node in data_spec['HeadNode']:
    head_node_type = head_node['name']
    if Node_Props['ImportStudyFile']:
        IMPORT_STUDY = True
        study_name = configuration_files['STUDY_FILE'] + data_spec['MockDataProps']["StudyName"] +  "-study.txt"
        study_df = import_study_file(study_name)
    else:
        IMPORT_STUDY = False
    if head_node_type in data_spec['IncludeNodes'].keys():
        logging.error('HeadNode ' + head_node_type + 'is in the IncludeNodes.')
        sys.exit()
    head_node_count = head_node['count']
    id_prefix = head_node['Prefix']
    dst_node_type = head_node_type
    # random a set of id without duplicate
    node_id_number_list = random.sample(range(10**5, 10**7), head_node_count + 1)
    head_node_index = 0
    for count in range(head_node_count):
        # node_id = id_prefix + "_" + str(random.randint(10**5, 10**6))
        
        if IMPORT_STUDY == True and head_node_type  == "study":
            field_name = id_field_data['Properties']['id_fields'][head_node_type]
            node_id = study_df.iloc[0][field_name]
        else: 
            node_id = id_prefix + "-" + str(node_id_number_list[head_node_index]).zfill(7)
            #node_id = id_prefix + "-" + str(node_id_number_list[head_node_index])# for bento
        
        head_node_index += 1
        parent_node_id_list = []
        child_node_id_list = []
        node_type = head_node_type
        node_attributes = {}
        data_node = DataNode(node_id = node_id, parent_node_id_list = parent_node_id_list, child_node_id_list = child_node_id_list,
                             node_type = node_type, node_attributes = {})
        dict_of_data_nodes[head_node_type].append(data_node)

edge_specs = data_spec['RelationshipSpecs']

In [92]:
for dst_node_type in edge_specs.keys():
    dst_data_nodes_list = dict_of_data_nodes[dst_node_type]

In [93]:
def findEdgeType(node_data, src_node_type, dst_node_type):
    for edge_type in node_data['Relationships']:
        for ends in node_data['Relationships'][edge_type]['Ends']:
            if ends['Src'] == src_node_type and ends['Dst'] == dst_node_type:
                return edge_type
    return None

In [94]:
#Function to create a skeleton data graph.
#Create a skeleton data graph.
def SpawnNodes():
    created_children = []
    children = []
    for dst_node_type in edge_specs.keys():
        print(f"working on { dst_node_type}")
        if dst_node_type in dict_of_data_nodes and dst_node_type not in created_children:
            dst_data_nodes_list = dict_of_data_nodes[dst_node_type]
            # for dst_data_node in dst_data_nodes_list:
            for src_node_type in edge_specs[dst_node_type].keys():
                print(f"check releationship: {dst_node_type}, {src_node_type}")
                node_counter = includeNodes[src_node_type]['NodeCount']
                node_distribution = edge_specs[dst_node_type][src_node_type]['SrcNodeCount']
                id_prefix = includeNodes[src_node_type]['Prefix']
                # random a set of id without duplicate
                node_id_number_list = random.sample(range(10**5, 10**7), node_counter)
                node_index = 0
                parent_node_index = 0
                parent_node_length = len(dst_data_nodes_list)
                if parent_node_length == 0:
                    continue
                
                step = int(node_counter / parent_node_length)
                # if the distribution is random
                # print(node_counter, parent_node_length)
                if node_distribution == 'random':
                    node_counter_list = range(node_counter)
                    random_split_points = random.sample(node_counter_list, parent_node_length - 1)
                    random_split_points.sort()
                    random_split_points_index = 0
                for count in range(node_counter):
                    if node_distribution == 'fixed':
                        if node_index % step == 0 and node_index != 0:
                            parent_node_index += 1
                    elif node_distribution == 'random':
                        #print(node_index)
                        if random_split_points_index < len(random_split_points):
                            if node_index == random_split_points[random_split_points_index]:
                                random_split_points_index += 1
                                parent_node_index += 1
                    if parent_node_index > parent_node_length - 1:
                            parent_node_index = parent_node_length - 1
                    # node_id = id_prefix + "_" + str(random.randint(10**5, 10**6))
                    #node_id = id_prefix + "-" + str(node_id_number_list[node_index])
                    node_id = id_prefix + "-" + str(node_id_number_list[node_index]).zfill(7)
                    if src_node_type not in children:
                        node_index += 1
                        parent_node_id_list = []
                        parent_node_id_list.append(dst_data_nodes_list[parent_node_index].node_id)
                        child_node_id_list = []
                        node_type = src_node_type
                        node_attributes = {}
                        src_data_node = DataNode(node_id = node_id, parent_node_id_list = parent_node_id_list, child_node_id_list = child_node_id_list,
                                             node_type = node_type, node_attributes = {}) #source node created.
                        dict_of_data_nodes[src_node_type].append(src_data_node) #source node added to the dict of nodes.

                        dst_data_nodes_list[parent_node_index].child_node_id_list.append(node_id) #add created source node to the child nodes list for dst node.
                    elif src_node_type in children:
                        dict_of_data_nodes[src_node_type][node_index].parent_node_id_list.append(dst_data_nodes_list[parent_node_index].node_id)
                        node_index += 1
                    edge_id = "edge" + "_" + str(random.randint(10**5, 10**6))
                    edge_type = findEdgeType(node_data, src_node_type, dst_node_type)
                    # edge_type = edge_specs[dst_node_type][src_node_type]['EdgeType']
                    edge_attributes = {}
                    data_edge = DataEdge(edge_id = edge_id, edge_type = edge_type, source_node = src_data_node, 
                                     destination_node = dst_data_nodes_list[parent_node_index], edge_attributes = edge_attributes) #edge created.
                    dict_of_data_edges[edge_id] = data_edge #edge added to the dict of edges.
                children.append(src_node_type)
            created_children.append(dst_node_type)
    data_graph = Graph(dict_of_data_nodes = dict_of_data_nodes, dict_of_data_edges = dict_of_data_edges)
    return data_graph

In [95]:
#Create skeleton data graph
data_graph = SpawnNodes()

working on study
check releationship: study, participant


In [96]:
#Examine skeleton data graph
# data_graph.summary()
summary = data_graph.summary()
pprint.pprint(summary)
print('END SPAWN SECTION')
######END SPAWN SECTION######

{'Edges Summary': {'associated_with': 92664},
 'Nodes Summary': {'participant': 97795, 'study': 1}}
END SPAWN SECTION


In [97]:
relationship_node_dict = {}
node_id_field_dict = {}
node_list = []
for parent_id in data_spec['RelationshipSpecs']:
    for node_id in data_spec['RelationshipSpecs'][parent_id]:
        relationship_node = {}
        relationship_node['parent_id'] = []
        relationship_node['parent_id_field'] = []
        relationship_node['node_id'] = node_id
        relationship_node['parent_id'].append(parent_id)
        if node_id in id_field_data['Properties']['id_fields']:
            relationship_node['node_id_field'] = id_field_data['Properties']['id_fields'][node_id]
        else:
            relationship_node['node_id_field'] ='node_id'
        if parent_id in id_field_data['Properties']['id_fields']:
            relationship_node['parent_id_field'].append(id_field_data['Properties']['id_fields'][parent_id])
        else:
            relationship_node['parent_id_field'].append('parent_id')
        if node_id not in relationship_node_dict.keys():
            relationship_node_dict[node_id] = relationship_node
        else:
            relationship_node_dict[node_id]['parent_id'].append(relationship_node['parent_id'][0])
            relationship_node_dict[node_id]['parent_id_field'].append(relationship_node['parent_id_field'][0])

for node_type in data_graph.dict_of_data_nodes:
    if node_type in id_field_data['Properties']['id_fields']:
        node_id_field_dict[node_type] = id_field_data['Properties']['id_fields'][node_type]

In [98]:
def GetParentIDField(node_type, index):
    if node_type in relationship_node_dict:
        if isinstance(relationship_node_dict[node_type]['parent_id_field'][index], list):
            return relationship_node_dict[node_type]['parent_id'][index]+'.'+relationship_node_dict[node_type]['parent_id_field'][index][0]
        else:
            return relationship_node_dict[node_type]['parent_id'][index]+'.'+relationship_node_dict[node_type]['parent_id_field'][index]
    else:
        return 'parent_id'
def GetNodeIDField(node_type):
    if node_type in node_id_field_dict:
        if isinstance(node_id_field_dict[node_type], list):
            return node_id_field_dict[node_type][0]
        else:
            return node_id_field_dict[node_type]
    else:
        return 'node_id'

In [99]:
######BEGIN FILL SECTION######
print('BEGIN FILL SECTION')
includePropsList = data_spec['IncludeProperties']

BEGIN FILL SECTION


In [100]:
data_graph.fill_graph(listOfProps = includePropsList, 
                      model_nodes_dict = dict_of_model_nodes, 
                      model_props_dict = dict_of_model_properties)         
                      #cancer_dict = cancer_data,
                      #tumor_dict = tumor_dict)
print('END FILL SECTION')
######END FILL SECTION######

Populating node_type for study
Populating node_type for participant
END FILL SECTION


In [101]:
######PRINT DATA FILES######
print('PRINT DATA FILES')
child_node_id_dict = {}
child_node_id_list = []
for node_type in data_graph.dict_of_data_nodes:
    node_values_dict = defaultdict(list)
    df = pd.DataFrame()
    position = 0
    for node in data_graph.dict_of_data_nodes[node_type]:

        node_values_dict['type'].append(node.node_type)
        if node.parent_node_id_list:
            index = 0
            for parent_node_id in node.parent_node_id_list:
                if node.node_id not in child_node_id_list:
                    node_values_dict[GetParentIDField(node_type, index)].append(parent_node_id) #parent
                else:
                    node_values_dict[GetParentIDField(node_type, index)].append(child_node_id_dict[node.node_id])
                    data_graph.dict_of_data_nodes[node_type][position].parent_node_id_list[0] = child_node_id_dict[node.node_id]
                index += 1
        
        if GetNodeIDField(node_type) not in node.node_attributes:
            node_values_dict[GetNodeIDField(node_type)].append(node.node_id) #node


        if GetNodeIDField(node_type) in node.node_attributes and GetNodeIDField(node_type) in synthetic_values_df.keys():
            res = synthetic_values_df[GetNodeIDField(node_type)].tolist()
            # res = synthetic_values_df[GetNodeIDField(node_type)].tolist()
            trim_res = [i for i in res if i]
            # if the node_type has more nodes than the values of all usable node_id
            if len(data_graph.dict_of_data_nodes[node_type]) > len(trim_res):
                error_message = 'node ' + node_type + ' is running out of all usable node_ids from ' + GetNodeIDField(node_type) + '.'
                # delete all previous generate tsv files
                mydir = os.path.abspath(Output_Folder)
                filelist = [ f for f in os.listdir(mydir) if f.endswith(".tsv") ]
                for f in filelist:
                    os.remove(os.path.join(mydir, f))
                sys.exit(error_message)
            new_node_id_list = []
            for new_node in data_graph.dict_of_data_nodes[node_type]:
                new_node_id_list.append(new_node.node_attributes[GetNodeIDField(node_type)])
            new_node_id_list_counter = Counter(new_node_id_list)
            reselect_value = False
            #Check if the new node_id list has duplicate node_id
            for value in new_node_id_list_counter.values():
                if value > 1:
                    reselect_value = True
            if reselect_value == True:
                new_value_list = random.sample(trim_res, len(new_node_id_list))
                for i in range(len(data_graph.dict_of_data_nodes[node_type])):
                    data_graph.dict_of_data_nodes[node_type][i].node_attributes[GetNodeIDField(node_type)] = new_value_list[i]
            data_graph.dict_of_data_nodes[node_type][position].node_id = data_graph.dict_of_data_nodes[node_type][position].node_attributes[GetNodeIDField(node_type)]
            for child_node_id in data_graph.dict_of_data_nodes[node_type][position].child_node_id_list:
                child_node_id_list.append(child_node_id)
                child_node_id_dict[child_node_id] = data_graph.dict_of_data_nodes[node_type][position].node_id
        if GetNodeIDField(node_type) in node.node_attributes and GetNodeIDField(node_type) not in synthetic_values_df.keys():
            # if the user adds the id field into the data spec document accidentally
            del node.node_attributes[GetNodeIDField(node_type)]
            node_values_dict[GetNodeIDField(node_type)].append(node.node_id) #node
        for node_prop in node.node_attributes:
            # print(node_type)
            # print(node.node_attributes)
            node_values_dict[node_prop].append(node.node_attributes[node_prop])
        position+=1
    print(f"working on {node_type} file")
    for node_values_key in node_values_dict:
        try:
            df[node_values_key] = node_values_dict[node_values_key]
        except Exception as e:
            print(node_values_key)
            
    if node_type == 'participant':
        control_part = df.query("participant_case_indicator == 'No'")
        df.loc[control_part.index,"cancer_diagnosis_primary_site"] = 'Not Applicable'
        df.loc[control_part.index,"cancer_diagnosis_disease_morphology"] = 'Not Applicable'
        df.loc[control_part.index,"age_at_first_cancer_diagnosis"] = np.nan
        
    
    #if "cancer_diagnosis_primary_site" in df.columns:
    #    cancer_location_df = all_terms [["cancer_diagnosis_primary_site", "cancer_diagnosis_subsite", "cancer_diagnosis_ICD-O_location"]]
    #    df = df.merge(cancer_location_df, how="left", left_on="cancer_diagnosis_primary_site", 
    #                  right_on="cancer_diagnosis_ICD-O_location")
        
        
    #file_index = 1
    #while len(df) > 0:
    #    if len(df) > 10000:
    #        temp_df = df[:10000]
    #    else:
    #        temp_df = df
    
    #file_name = configuration_files['OUTPUT_FOLDER'] + node_type + "_" + str(file_index) + ".tsv"
    file_name = Output_Folder + file_sep + node_type + ".tsv"
    if not os.path.exists(Output_Folder):
        os.mkdir(Output_Folder)
    if IMPORT_STUDY == True and node_type == 'study':
        file_name =  Output_Folder + file_sep + data_spec['MockDataProps']["StudyName"] +  "-study.txt"
        study_df = import_study_file(study_name)
        study_df.to_csv(file_name, sep = "\t", index = False)
    else:
        print(f"Writting file: {file_name}")
        df.to_csv(file_name, sep = "\t", index = False)
        #temp_df.to_csv(file_name, sep = "\t", index = False)
    #file_index = file_index + 1
    #df = df[10000:]

PRINT DATA FILES
working on study file
working on participant file
Writting file: .\Mock_Data_Output\Study_BSCT\participant.tsv


In [102]:
df

,type,study.study_short_name,participant_id,participant_case_indicator,cancer_diagnosis_primary_site,cancer_diagnosis_disease_morphology,age_at_enrollment,age_at_first_cancer_diagnosis,participant_race,participant_ethnicity,participant_sex,participant_ncbi_taxonomy_id
0,participant,BSCT,POPSCI-PARTICIPANT-3827705,Yes,C15.9,9987/3,19172.0,30081.0,Native Hawaiian or other Pacific Islander,Not Hispanic or Latino,Male,9606
1,participant,BSCT,POPSCI-PARTICIPANT-1901975,No,Not Applicable,Not Applicable,24624.0,NaN,White,Not Hispanic or Latino,Male,9606
2,participant,BSCT,POPSCI-PARTICIPANT-1970503,Yes,C32.3,9504/3,15825.0,21343.0,White,Not Hispanic or Latino,Female,9606
3,participant,BSCT,POPSCI-PARTICIPANT-8593502,No,Not Applicable,Not Applicable,31307.0,NaN,White,Hispanic or Latino,Female,9606
4,participant,BSCT,POPSCI-PARTICIPANT-8139229,Yes,C30.0,8982/3,12181.0,17674.0,White,Not Hispanic or Latino,Male,9606
...,...,...,...,...,...,...,...,...,...,...,...,...
97790,participant,BSCT,POPSCI-PARTICIPANT-2693083,No,Not Applicable,Not Applicable,13987.0,NaN,White,Not Hispanic or Latino,Female,9606
97791,participant,BSCT,POPSCI-PARTICIPANT-8885424,Yes,C67.4,9911/3,18481.0,24157.0,Black or African American | White,Not Hispanic or Latino,Female,9606
97792,participant,BSCT,POPSCI-PARTICIPANT-8775941,Yes,C02.1,9986/3,30370.0,31664.0,White,Hispanic or Latino,Male,9606
97793,participant,BSCT,POPSCI-PARTICIPANT-0940551,Yes,C54.9,9680/3,32169.0,28704.0,White,Not Hispanic or Latino,Female,9606


In [103]:
#import sys
#import os
#sys.path.append(os.path.abspath(r'C:\Program Files\Spyder\Python\Lib\site-packages'))

In [104]:
######VALIDATE DATA FILES######
#print('VALIDATE DATA FILES')
#from data_loader import DataLoader
#from icdc_schema import ICDC_Schema
#from neo4j import GraphDatabase
#from props import Props

In [ ]:
#file_list = [f for f in os.listdir(configuration_files['OUTPUT_FOLDER']) if f.endswith('.tsv')]
#for i in range(0, len(file_list)):
#    file_list[i] = configuration_files['OUTPUT_FOLDER'] + file_list[i]
#props = Props(configuration_files['ID_FILE'])
#schema = ICDC_Schema([configuration_files['NODE_FILE'], configuration_files['PROP_FILE']], props)
#loader = DataLoader(None, schema)
#fileValidationResult = loader.validate_files(False, file_list, 0)

In [ ]:
def relationshipValidation(dict_of_data_edges, node_data, includeNodes):
    for edge in dict_of_data_edges.values():
        # print(edge)
        mul = node_data['Relationships'][edge.edge_type]['Mul']
        prefix = includeNodes[edge.source_node.node_type]['Prefix']
        child_node_id_list = []
        for child_node_id in edge.destination_node.child_node_id_list:
            for prefix in child_node_id:
                child_node_id_list.append(child_node_id)
        if mul == 'one_to_one' and len(child_node_id_list) > 1:
            logging.error(edge.source_node.node_type + ' ' + 'one_to_one relationship failed, parent already has a child!')
            return False
    return True

In [ ]:
#relationshipValidationResult = relationshipValidation(dict_of_data_edges, node_data, includeNodes)
#if not relationshipValidationResult or not fileValidationResult:
#    print('Validation fail, delete all files inside the data folder.')
#    mydir = os.path.abspath(configuration_files['OUTPUT_FOLDER'])
#    filelist = [ f for f in os.listdir(mydir) if f.endswith(".tsv") ]
#    for f in filelist:
#        os.remove(os.path.join(mydir, f))
#else:
#    print('Validation success')